# Testing the Globus API

## Define variables

In [1]:
CLIENT_ID = 'e8954f4a-23c6-4ffc-8eaa-747fda1f9351'
blueTeamEndpointID = 'eed5aba8-6bc9-11e7-a9c6-22000bf2d287'
redTeamEndpointID = '3435b504-6bc4-11e7-a9c6-22000bf2d287'
rootTransferFolderName = "Red_Team_Stuff"
transferFile = 'Transparent Aluminum.zip'
token_path = "./.tokens.json"

## Import Globus,  connect to app and start TransferClient

In [2]:
import os
import json
import globus_sdk

client = globus_sdk.NativeAppAuthClient(CLIENT_ID)

if os.path.exists(token_path):
    with open(token_path) as tok_file:
        tokens = json.load(tok_file)
else:
    client.oauth2_start_flow(refresh_tokens=True)

    print('Please go to this URL and login: {0}'
          .format(client.oauth2_get_authorize_url()))

    get_input = getattr(__builtins__, 'raw_input', input)
    auth_code = get_input('Please enter the code here: ').strip()
    token_response = client.oauth2_exchange_code_for_tokens(auth_code)

    # let's get stuff for the Globus Transfer service
    globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']
    # the refresh token and access token, often abbr. as RT and AT
#    transfer_rt = globus_transfer_data['refresh_token']
#    transfer_at = globus_transfer_data['access_token']
#    expires_at_s = globus_transfer_data['expires_at_seconds']
    
    with open(token_path, "w") as tok_file:
        json.dump(globus_transfer_data, tok_file)
    tokens = globus_transfer_data

authorizer = globus_sdk.RefreshTokenAuthorizer(
    tokens["refresh_token"], client, access_token=tokens["access_token"], expires_at=tokens["expires_at_seconds"])

# and try using `tc` to make TransferClient calls.
tc = globus_sdk.TransferClient(authorizer=authorizer)

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=e8954f4a-23c6-4ffc-8eaa-747fda1f9351&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=3XKfTOmjpEVptOcB8DcMEsGh2uUZEs2i3Bk73wZqq6U&code_challenge_method=S256&access_type=offline
Please enter the code here: JqU2uudW9f6oU4DToiWr9Wmz1Q9fSa


## Get a response from the endpoint

In [4]:
endpoint = tc.get_endpoint(blueTeamEndpointID)
print("Endpoint name: {}".format(endpoint["canonical_name"]))

Endpoint name: u_cdtg35asyreyhnmqmhbzl5jab4#mdi_summit_blue_team


## List dirs and files in the Blue Team endpoint

In [6]:
blueTeamDirs = []
for response in tc.operation_ls(blueTeamEndpointID):
    print('Name: {}, type: {}'.format(response['name'], response['type']))
    if response['type'] == 'dir':
        blueTeamDirs.append(response)

TransferAPIError: (400, 'ClientError.ActivationRequired', "The endpoint 'eed5aba8-6bc9-11e7-a9c6-22000bf2d287' is not activated", '1Iw42oXtm')

## Check if root transfer path exist otherwise create it

In [5]:
newRootPath = "./{}".format(rootTransferFolderName)
rootPathExists = False
for folder in blueTeamDirs:
    if folder['name'] == rootTransferFolderName:
        rootPathExists = True
if not rootPathExists: #Create an new dir in the blue team endpoint
    mkDirResponse = tc.operation_mkdir(blueTeamEndpointID, path=newRootPath)
    print mkDirResponse
else:
    print 'Root transfer path already exists'

Root transfer path already exists


## Create new dir in the root transfer dir with dt stamp

In [6]:
from time import gmtime, strftime
dtStamp = strftime("%Y-%m-%d %H:%M:%S", gmtime())
newTransferFolderName = dtStamp.replace(' ','_')
newFolderPath = '{}/{}'.format(newRootPath, newTransferFolderName)
mkDirResponse = tc.operation_mkdir(blueTeamEndpointID, path=newFolderPath)
print mkDirResponse

{
  "message": "The directory was created successfully", 
  "code": "DirectoryCreated", 
  "resource": "/operation/endpoint/eed5aba8-6bc9-11e7-a9c6-22000bf2d287/mkdir", 
  "DATA_TYPE": "mkdir_result", 
  "request_id": "kTJP00yrm"
}


## Transfer file to new folder

In [7]:
transferData = globus_sdk.TransferData(tc, redTeamEndpointID,
                                blueTeamEndpointID)
transferedFilePath = '{}/{}'.format(newFolderPath, transferFile)
transferData.add_item(transferFile, transferedFilePath)
transfer_result = tc.submit_transfer(transferData)
print transfer_result

{
  "submission_id": "c0a7e2a7-6c95-11e7-a9cb-22000bf2d287", 
  "code": "Accepted", 
  "resource": "/transfer", 
  "task_id": "c0a7e2a6-6c95-11e7-a9cb-22000bf2d287", 
  "DATA_TYPE": "transfer_result", 
  "request_id": "mTfUyuXlR", 
  "message": "The transfer has been accepted and a task has been created and queued for execution", 
  "task_link": {
    "href": "task/c0a7e2a6-6c95-11e7-a9cb-22000bf2d287?format=json", 
    "resource": "task", 
    "rel": "related", 
    "DATA_TYPE": "link", 
    "title": "related task"
  }
}


## Example - delete dir in endpoint

In [ ]:
deleteData = globus_sdk.DeleteData(tc, blueTeamEndpointID, recursive=True)
deleteData.add_item("Red_Team_Stuff")
delete_result = tc.submit_delete(deleteData)
print("task_id =", delete_result["task_id"])

## Example - text search on Globus endpoints

In [ ]:
for ep in tc.endpoint_search('Jan'):
    print(ep['display_name'])

## Example - Rename the new dir

In [ ]:
renamePathResponse = tc.operation_rename(blueTeamEndpointID, oldpath="./Jan's Folder", newpath="./Red_Team_Stuff")
print renamePathResponse